In [1]:
import numpy as np
import tensorflow as tf
from tf_agents.networks.q_network import QNetwork
import tf_agents.networks.network as network
from tf_agents.specs import tensor_spec
from snake_game import SnakeGame
from scene import Scene
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D

scene = Scene(init_randomly=True)
episodes_count = 5000


In [2]:
from tf_agents.environments.tf_py_environment import TFPyEnvironment

env = SnakeGame(scene)
env = TFPyEnvironment(env)


In [3]:
conv_layer_params = [(32, (2, 2), 1), (64, (2, 2), 1)]
fc_layer_params = [128, 16]

q_net = QNetwork(
    env.observation_spec(),
    env.action_spec(),
    conv_layer_params=conv_layer_params,
    fc_layer_params=fc_layer_params)


In [3]:
class MyQNetwork(network.Network):
    def __init__(self,
                 input_tensor_spec,
                 action_spec,
                 name='MyQNetwork'):
        super(MyQNetwork, self).__init__(
            input_tensor_spec=input_tensor_spec,
            state_spec=(),
            name=name)
        
        self._action_spec = action_spec

        input_shape = tf.expand_dims(input_tensor_spec, axis=0)
        # Create your custom model
        self._model = Sequential([
            Conv2D(32, (2, 2), 1, input_shape=input_shape, activation='relu', kernel_initializer='he_normal'),
            Conv2D(64, (2, 2), 1, activation='relu', kernel_initializer='he_normal'),
            Flatten(),
            Dense(128, activation='relu', kernel_initializer='he_normal'),
            Dense(16, activation='relu', kernel_initializer='he_normal'),
            Dense(4)
        ])

    def call(self, observations, step_type=None, network_state=(), training=False):
        output = self._model(observations)
        return output, network_state

    def create_variables(self, input_tensor_spec, **kwargs):
        self(tf.zeros(input_tensor_spec.shape, input_tensor_spec.dtype))

    def copy(self, name=''):
        pass

q_net = MyQNetwork(
    env.observation_spec(),
    env.action_spec())


ValueError: Attempt to convert a value (BoundedTensorSpec(shape=(5, 5, 4), dtype=tf.float32, name='observation', minimum=array([0., 0., 0., 0.], dtype=float32), maximum=array([1., 1., 1., 1.], dtype=float32))) with an unsupported type (<class 'tensorflow.python.framework.tensor.BoundedTensorSpec'>) to a Tensor.

In [32]:
# Create the agent
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tensorflow import keras
from keras.optimizers.legacy import Adam
from keras.losses import Huber
from tf_agents.trajectories import TimeStep

train_step_counter = tf.Variable(0)
optimizer = Adam(learning_rate=0.003)

# epsilon = lambda train_step: (1 / (tf.cast(tf.linspace(1, 8, episodes_count)**3, tf.float32)))[train_step]
epsilon = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0,
    decay_steps=episodes_count,
    end_learning_rate=0.01
)

agent = DqnAgent(
    time_step_spec=env.time_step_spec(),
    action_spec=env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=Huber(reduction="none"),
    gamma=tf.constant(0.95, dtype=tf.float32),
    train_step_counter=train_step_counter,
    epsilon_greedy=lambda: epsilon(train_step_counter)
)


ValueError: Exception encountered when calling layer 'MyQNetwork' (type MyQNetwork).

Input 0 of layer "sequential" is incompatible with the layer: expected shape=(None, 5, 5, 4), found shape=(5, 5, 4)

Call arguments received by layer 'MyQNetwork' (type MyQNetwork):
  • observations=tf.Tensor(shape=(5, 5, 4), dtype=float32)
  • step_type=None
  • network_state=()
  • training=False
  In call to configurable 'DqnAgent' (<class 'tf_agents.agents.dqn.dqn_agent.DqnAgent'>)

In [5]:
# Create the replay buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size,
    max_length=100000
)


In [6]:
# Create the training metrics
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric()
]


In [7]:
# Create the driver
from tf_agents.drivers import dynamic_step_driver

collect_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=agent.collect_policy,
    observers=[replay_buffer.add_batch] + train_metrics,
    num_steps=4
)

# Run a random policy to fill the replay buffer
from tf_agents.policies.random_tf_policy import RandomTFPolicy

initial_collect_policy = RandomTFPolicy(env.time_step_spec(), env.action_spec())
init_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=initial_collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=20000
)

final_time_step, final_policy_state = init_driver.run()


In [8]:
# Create the dataset
dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2, # Capaz cambiar esto
    num_parallel_calls=3
).prefetch(3)


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [9]:
# Create the training loop

from tf_agents.utils.common import function
from tf_agents.trajectories import trajectory
from tf_agents.eval.metric_utils import log_metrics
import logging

logging.getLogger().setLevel(logging.INFO)

collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

def train_agent(n_iterations):
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(env.batch_size)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        if iteration % 1000 == 0:
            print("Iteration: ", iteration)
            log_metrics(train_metrics)


In [10]:
train_agent(episodes_count)


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 4
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


Iteration:  0


INFO:absl: 
		 NumberOfEpisodes = 3037
		 EnvironmentSteps = 4004
		 AverageReturn = -44988.69921875
		 AverageEpisodeLength = 0.699999988079071


Iteration:  1000


...

Iteration:  147000


INFO:absl: 
		 NumberOfEpisodes = 181980
		 EnvironmentSteps = 592004
		 AverageReturn = -1001019.875
		 AverageEpisodeLength = 25.5


Iteration:  148000


INFO:absl: 
		 NumberOfEpisodes = 182163
		 EnvironmentSteps = 596004
		 AverageReturn = -978691.1875
		 AverageEpisodeLength = 29.399999618530273


Iteration:  149000


In [11]:
# Evaluate the agent

time_step = env.reset()
rewards = []
steps = 0

while not time_step.is_last() and steps < 200:
    steps += 1
    print("Map:\n", np.argmax(time_step.observation.numpy(), axis=3))
    action_step = agent.policy.action(time_step)
    print("Action: ", action_step.action.numpy())
    time_step = env.step(action_step.action)
    rewards.append(time_step.reward.numpy()[0])
    
print("Total reward: ", sum(rewards))
print("Total steps: ", steps)


Map:
 [[[0 0 0 0 0]
  [0 1 0 0 0]
  [0 2 0 0 0]
  [0 0 0 3 0]
  [0 0 0 0 0]]]
Action:  [2]
Map:
 [[[0 0 0 0 0]
  [0 2 1 0 0]
  [0 0 0 0 0]
  [0 0 0 3 0]
  [0 0 0 0 0]]]
Action:  [2]
Map:
 [[[0 0 0 0 0]
  [0 0 2 1 0]
  [0 0 0 0 0]
  [0 0 0 3 0]
  [0 0 0 0 0]]]
Action:  [3]
Map:
 [[[0 0 0 0 0]
  [0 0 0 2 0]
  [0 0 0 1 0]
  [0 0 0 3 0]
  [0 0 0 0 0]]]
Action:  [3]
Map:
 [[[0 0 0 0 0]
  [0 0 0 2 3]
  [0 0 0 2 0]
  [0 0 0 1 0]
  [0 0 0 0 0]]]
Action:  [2]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 3]
  [0 0 0 2 0]
  [0 0 0 2 1]
  [0 0 0 0 0]]]
Action:  [1]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 3]
  [0 0 0 0 1]
  [0 0 0 2 2]
  [0 0 0 0 0]]]
Action:  [1]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 1]
  [0 0 0 0 2]
  [0 0 0 2 2]
  [0 0 0 0 3]]]
Action:  [0]
Map:
 [[[0 0 0 0 0]
  [0 0 0 1 2]
  [0 0 0 0 2]
  [0 0 0 0 2]
  [0 0 0 0 3]]]
Action:  [3]
Map:
 [[[0 0 0 0 0]
  [0 0 0 2 2]
  [0 0 0 1 2]
  [0 0 0 0 0]
  [0 0 0 0 3]]]
Action:  [3]
Map:
 [[[0 0 0 0 0]
  [0 0 0 2 2]
  [0 0 0 2 0]
  [0 0 0 1 0]
  [0 0 0 0 3]]]
Action:  [2]

In [12]:
# Count the average steps per episode

n = 200
total_steps = 0
total_reward = 0
steps = 0
total_500 = 0

for i in range(n):
    print("Episode: ", i)
    time_step = env.reset()
    steps = 0
    while not time_step.is_last() and steps < 500:
        steps += 1
        action_step = agent.policy.action(time_step)
        time_step = env.step(action_step.action)
        reward = time_step.reward.numpy()[0]
        total_reward += reward
    
    if steps == 500:
        total_500 += 1
    else:
        total_steps += steps
        
if steps == 500:
    print("500 episodes reached")
    
print("Average steps per episode: ", total_steps / (n - total_500))
print("Average reward per episode: ", total_reward / n)
print("Total 500 episodes: ", total_500)


Episode:  0
Episode:  1
Episode:  2
Episode:  3
Episode:  4
Episode:  5
Episode:  6
Episode:  7
Episode:  8
Episode:  9
Episode:  10
Episode:  11
Episode:  12
Episode:  13
Episode:  14
Episode:  15
Episode:  16
Episode:  17
Episode:  18
Episode:  19
Episode:  20
Episode:  21
Episode:  22
Episode:  23
Episode:  24
Episode:  25
Episode:  26
Episode:  27
Episode:  28
Episode:  29
Episode:  30
Episode:  31
Episode:  32
Episode:  33
Episode:  34
Episode:  35
Episode:  36
Episode:  37
Episode:  38
Episode:  39
Episode:  40
Episode:  41
Episode:  42
Episode:  43
Episode:  44
Episode:  45
Episode:  46
Episode:  47
Episode:  48
Episode:  49
Episode:  50
Episode:  51
Episode:  52
Episode:  53
Episode:  54
Episode:  55
Episode:  56
Episode:  57
Episode:  58
Episode:  59
Episode:  60
Episode:  61
Episode:  62
Episode:  63
Episode:  64
Episode:  65
Episode:  66
Episode:  67
Episode:  68
Episode:  69
Episode:  70
Episode:  71
Episode:  72
Episode:  73
Episode:  74
Episode:  75
Episode:  76
Episode: 

In [13]:
print("Map:\n", np.argmax(time_step.observation.numpy(), axis=3))


Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [3 0 0 0 0]
  [1 2 0 0 0]]]


In [14]:
action_step = agent.policy.action(time_step)
time_step = env.step(action_step.action)
print("Map:\n", np.argmax(time_step.observation.numpy(), axis=3))


Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 3]
  [1 0 0 0 0]
  [2 2 0 0 0]]]


In [15]:
# Save the agent
import os
from tf_agents.policies.policy_saver import PolicySaver

saver = PolicySaver(agent.policy, batch_size=None)
saver.save("policy")


INFO:absl:Function `function_with_signature` contains input name(s) 0/step_type, 0/reward, 0/discount, 0/observation, 10066885, 10066887, 10066889, 10066891, 10066893, 10066895, 10066897, 10066899, 10066901, 10066903 with unsupported characters which will be renamed to step_type, reward, discount, observation, unknown, unknown_0, unknown_1, unknown_2, unknown_3, unknown_4, unknown_5, unknown_6, unknown_7, unknown_8 in the SavedModel.
INFO:absl:Function `function_with_signature` contains input name(s) 10066948 with unsupported characters which will be renamed to unknown in the SavedModel.


INFO:tensorflow:Assets written to: policy/assets


/Users/juanpablo/miniconda3/lib/python3.11/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:475: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
INFO:tensorflow:Assets written to: policy/assets
